In [9]:
import pandas as pd
import numpy as np
import gzip
import os
import warnings

# --- Import ML / Stats Libraries ---
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import ColumnTransformer
from lifelines import CoxPHFitter

In [10]:
# ==============================================================================
# --- 1. DEFINE FILE PATHS ---
# ==============================================================================
print("--- Step 1: Defining File Paths ---")
# Get the base data directory
base_data_dir = "/Users/saifahmed/development/ESCC/data"

# Path to the final processed genomic matrix
genomic_matrix_file = os.path.join(base_data_dir, "GSE53622_series_matrix.txt.gz")

# Path to the clinical data file
clinical_file = os.path.join(base_data_dir, "GSE53622_clinical_data_of_patients_independent_set.xlsx")

print(f"Genomic file: {genomic_matrix_file}")
print(f"Clinical file: {clinical_file}")

--- Step 1: Defining File Paths ---
Genomic file: /Users/saifahmed/development/ESCC/data/GSE53622_series_matrix.txt.gz
Clinical file: /Users/saifahmed/development/ESCC/data/GSE53622_clinical_data_of_patients_independent_set.xlsx


In [13]:
# ==============================================================================
# --- 2. LOAD & INTEGRATE DATA (Steps 3, 4, 5) ---
# ==============================================================================
print("\n--- Step 2: Loading and Integrating Data ---")

# --- Load Genomic Data (G) ---
print("Loading genomic matrix... (this may take a moment)")
skip_n = 0
try:
    with gzip.open(genomic_matrix_file, 'rt', encoding='latin-1') as f:
        for i, line in enumerate(f):
            if line.startswith('!series_matrix_table_begin'):
                skip_n = i + 1
                break
except FileNotFoundError:
    print(f"FATAL ERROR: Genomic file not found at {genomic_matrix_file}")
    
# Suppress the harmless DtypeWarning (we fix this later)
warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)
    
genomic_matrix_df = pd.read_csv(
    genomic_matrix_file,
    sep='\t',
    compression='gzip',
    skiprows=skip_n,
    encoding='latin-1'
)
genomic_matrix_df = genomic_matrix_df.set_index('ID_REF')
genomic_matrix_df = genomic_matrix_df.dropna()
genomic_matrix_df = genomic_matrix_df.apply(pd.to_numeric, errors='coerce')
print(f"Loaded genomic data. Shape: {genomic_matrix_df.shape} (Probes x All Samples)")

# --- Load Clinical Data (C) ---
print("Loading clinical data...")
try:
    clinical_df = pd.read_excel(clinical_file, engine="openpyxl")
except FileNotFoundError:
    print(f"FATAL ERROR: Clinical file not found at {clinical_file}")
except ImportError:
    print("FATAL ERROR: 'openpyxl' library not found. Please run: !pip install openpyxl")
    
print(f"Loaded clinical data. Shape: {clinical_df.shape} (Patients x Clinical Features)")

# --- Build Full Patient Map & Filter (NEW, CORRECTED PARSER) ---
print("Building full GSM-to-Patient map from genomic file header...")
gsm_ids = []
sample_titles = []
try:
    with gzip.open(genomic_matrix_file, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Sample_geo_accession"):
                # Split by tab, discard 1st item, strip quotes
                gsm_ids = [item.strip('"') for item in line.strip().split('\t')[1:]]
            
            elif line.startswith("!Sample_title"):
                # Split by tab, discard 1st item, strip quotes
                sample_titles = [item.strip('"') for item in line.strip().split('\t')[1:]]

            elif line.startswith('!series_matrix_table_begin'):
                # We've found all the metadata we need
                break
except Exception as e:
    print(f"FATAL ERROR: Failed to read header: {e}")

# --- Now, build the map by zipping the two lists ---
full_data_mapping = {}
if len(gsm_ids) > 0 and len(gsm_ids) == len(sample_titles):
    for i in range(len(gsm_ids)):
        gsm_id = gsm_ids[i]
        title = sample_titles[i]
        
        try:
            patient_id = "ec" + title.split('patient ')[-1]
            tissue_type = "cancer" if "cancer" in title else "normal"
            full_data_mapping[gsm_id] = {
                'patient_id': patient_id,
                'tissue_type': tissue_type
            }
        except Exception as e:
            print(f"Warning: Could not parse title: {title}")
            
    print(f"Full map built. Found {len(full_data_mapping)} total samples.")
else:
    print(f"FATAL ERROR: Header parsing failed. Found {len(gsm_ids)} GSMs and {len(sample_titles)} titles.")

# --- Filter for CANCER samples only ---
available_gsm_ids = genomic_matrix_df.columns
cancer_gsm_ids = []
patient_id_map = {}
for gsm_id in available_gsm_ids:
    if gsm_id in full_data_mapping and full_data_mapping[gsm_id]['tissue_type'] == 'cancer':
        cancer_gsm_ids.append(gsm_id)
        patient_id_map[gsm_id] = full_data_mapping[gsm_id]['patient_id']

genomic_cancer_df = genomic_matrix_df[cancer_gsm_ids]
print(f"Filtered for {len(cancer_gsm_ids)} 'cancer' samples. Discarding 'normal' samples.")

# --- Transpose & Merge ---
genomic_cancer_transposed_df = genomic_cancer_df.T
genomic_cancer_transposed_df = genomic_cancer_transposed_df.rename(index=patient_id_map)
genomic_cancer_transposed_df.index.name = 'Patient ID'

if 'Patient ID' in clinical_df.columns:
    clinical_df = clinical_df.set_index('Patient ID')
else:
    print("FATAL ERROR: 'Patient ID' column not found in clinical file.")

master_df_FULL = pd.merge(
    genomic_cancer_transposed_df,
    clinical_df,
    left_index=True,
    right_index=True,
    how="inner" 
)
print(f"Full Master DataFrame created. Shape: {master_df_FULL.shape}")


--- Step 2: Loading and Integrating Data ---
Loading genomic matrix... (this may take a moment)
Loaded genomic data. Shape: (71584, 120) (Probes x All Samples)
Loading clinical data...
Loaded clinical data. Shape: (60, 16) (Patients x Clinical Features)
Building full GSM-to-Patient map from genomic file header...
Full map built. Found 120 total samples.
Filtered for 60 'cancer' samples. Discarding 'normal' samples.
Full Master DataFrame created. Shape: (60, 71599)


In [14]:
# ==============================================================================
# --- 3. SEPARATE X and y (Step 6) ---
# ==============================================================================
print("\n--- Step 3: Separating X (Features) and y (Targets) ---")
target_event_col = 'Death at FU'
target_time_col = 'Survival time(months)'
clinical_feature_cols = [
    'Age', 'Sex', 'Tobacco use', 'Alcohol use', 'Tumor location',
    'Tumor grade', 'T stage', 'N stage', 'TNM stage', 'Arrhythmia',
    'Pneumonia', 'Anastomotic leak', 'Adjuvant therapy'
]

# Create y_survival_FULL
le = LabelEncoder()
y_event_encoded = le.fit_transform(master_df_FULL[target_event_col])

# More robust check for 'yes' (death) vs 'no' (alive)
death_event_label = le.transform(['yes'])[0]
if death_event_label == 1:
    print("Survival Event: 'yes' (death) = 1, 'no' (alive) = 0. (Correct)")
    y_event = y_event_encoded
else:
    print("Survival Event: 'yes' (death) = 0, 'no' (alive) = 1. (Flipping labels)")
    y_event = 1 - y_event_encoded

y_survival_FULL = pd.DataFrame({
    'event': y_event,
    'time': master_df_FULL[target_time_col]
}, index=master_df_FULL.index)

# Create X_clinical_FULL
X_clinical_FULL = master_df_FULL[clinical_feature_cols].copy()

# Create X_genomic_FULL
all_master_cols = master_df_FULL.columns
genomic_feature_cols = [
    col for col in all_master_cols
    if col not in clinical_feature_cols and col not in [target_event_col, target_time_col]
]
X_genomic_FULL = master_df_FULL[genomic_feature_cols].copy()

print(f"y_survival_FULL shape: {y_survival_FULL.shape} (Our 'y' target)")
print(f"X_clinical_FULL shape: {X_clinical_FULL.shape} (Our clinical 'X' features)")
print(f"X_genomic_FULL shape: {X_genomic_FULL.shape} (Our genomic 'X' features)")


--- Step 3: Separating X (Features) and y (Targets) ---
Survival Event: 'yes' (death) = 1, 'no' (alive) = 0. (Correct)
y_survival_FULL shape: (60, 2) (Our 'y' target)
X_clinical_FULL shape: (60, 13) (Our clinical 'X' features)
X_genomic_FULL shape: (60, 71584) (Our genomic 'X' features)


In [15]:
# ==============================================================================
# --- 4. FEATURE REDUCTION (Steps 7 & 8) ---
# ==============================================================================
print("\n--- Step 4: Starting Feature Reduction ---")

# --- Step 7: Low-Variance Filter (Unsupervised) ---
print("Running VarianceThreshold to remove 'flat-line' features...")
X_genomic_FULL.columns = X_genomic_FULL.columns.astype(str) 
var_filter = VarianceThreshold(threshold=0.1) 
X_genomic_filtered = var_filter.fit_transform(X_genomic_FULL)
retained_cols = var_filter.get_feature_names_out()
X_genomic_filtered_df_FULL = pd.DataFrame(
    X_genomic_filtered, 
    columns=retained_cols, 
    index=X_genomic_FULL.index
)
print(f"Features REMOVED (low variance): {X_genomic_FULL.shape[1] - X_genomic_filtered_df_FULL.shape[1]}")
print(f"Features RETAINED: {X_genomic_filtered_df_FULL.shape[1]}")

# --- Step 8: CoxPH Feature Selection (Supervised) ---
print("\n--- Starting CoxPH Feature Selection. ---")
print("--- THIS IS THE LONG STEP: IT WILL TAKE 30-60+ MINUTES. ---")
print(f"--- Fitting {X_genomic_filtered_df_FULL.shape[1]} individual models... ---")

feature_p_values = []
data_for_survival = y_survival_FULL.copy()
warnings.filterwarnings("ignore") # Suppress convergence warnings

for feature in X_genomic_filtered_df_FULL.columns:
    data_for_survival[feature] = X_genomic_filtered_df_FULL[feature]
    cph = CoxPHFitter()
    try:
        cph.fit(data_for_survival, duration_col='time', event_col='event')
        feature_p_values.append({
            'feature': feature,
            'p_value': cph.summary.loc[feature, 'p']
        })
    except Exception as e:
        pass # Skip features that fail to converge
    
    data_for_survival = data_for_survival.drop(columns=[feature])

print("--- CoxPH fitting complete. ---")

p_values_df = pd.DataFrame(feature_p_values)
p_values_df = p_values_df.sort_values(by='p_value').set_index('feature')

# --- Select the Top Features ---
significant_features = p_values_df[p_values_df['p_value'] < 0.05].index
print(f"Found {len(significant_features)} features with p-value < 0.05.")

if len(significant_features) < 100:
    print(f"Found fewer than 100. Selecting Top 100 features by p-value.")
    top_features = p_values_df.head(100).index
else:
    top_features = significant_features

X_genomic_final_FULL = X_genomic_filtered_df_FULL[top_features]
print(f"Successfully selected {len(top_features)} final genomic features.")
print(f"Final genomic data shape: {X_genomic_final_FULL.shape}")


--- Step 4: Starting Feature Reduction ---
Running VarianceThreshold to remove 'flat-line' features...
Features REMOVED (low variance): 5650
Features RETAINED: 65934

--- Starting CoxPH Feature Selection. ---
--- THIS IS THE LONG STEP: IT WILL TAKE 30-60+ MINUTES. ---
--- Fitting 65934 individual models... ---
--- CoxPH fitting complete. ---
Found 2805 features with p-value < 0.05.
Successfully selected 2805 final genomic features.
Final genomic data shape: (60, 2805)


In [16]:
# ==============================================================================
# --- 5. FINAL PREPROCESSING & COMBINATION (Step 9) ---
# ==============================================================================
print("\n--- Step 5: Preprocessing Clinical Data & Creating Final Matrix ---")

numerical_features = ['Age']
categorical_features = [
    'Sex', 'Tobacco use', 'Alcohol use', 'Tumor location',
    'Tumor grade', 'T stage', 'N stage', 'TNM stage', 'Arrhythmia',
    'Pneumonia', 'Anastomotic leak', 'Adjuvant therapy'
]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ],
    remainder='passthrough' 
)

X_clinical_processed = preprocessor.fit_transform(X_clinical_FULL)
cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
all_clinical_feature_names = numerical_features + list(cat_feature_names)

X_clinical_processed_df_FULL = pd.DataFrame(
    X_clinical_processed,
    columns=all_clinical_feature_names,
    index=X_clinical_FULL.index
)
print(f"Processed clinical data shape: {X_clinical_processed_df_FULL.shape}")

# --- Create the FINAL Input Matrix ---
X_final_FULL = pd.concat([X_genomic_final_FULL, X_clinical_processed_df_FULL], axis=1)

print("\n\n--- !!! DATA PREPARATION COMPLETE !!! ---")
print(f"Final Input (X_final_FULL) shape: {X_final_FULL.shape}")
print(f"Final Target (y_survival_FULL) shape: {y_survival_FULL.shape}")
print("\n--- Your model-ready data is now in X_final_FULL and y_survival_FULL ---")
print("--- You can now proceed to model training. ---")


--- Step 5: Preprocessing Clinical Data & Creating Final Matrix ---
Processed clinical data shape: (60, 33)


--- !!! DATA PREPARATION COMPLETE !!! ---
Final Input (X_final_FULL) shape: (60, 2838)
Final Target (y_survival_FULL) shape: (60, 2)

--- Your model-ready data is now in X_final_FULL and y_survival_FULL ---
--- You can now proceed to model training. ---


In [17]:
import os

# --- Step 10 (Corrected): Save Your Final Data Blocks ---
print("\n--- Step 10: Saving final, model-ready data to disk... ---")

# Define where to save the files
base_data_dir = "/Users/saifahmed/development/ESCC/data"
output_dir = os.path.join(base_data_dir, "processed")
os.makedirs(output_dir, exist_ok=True) # Create 'processed' folder if it doesn't exist

# --- Define the file paths ---
# 1. For the Clinical-Only Model
x_clinical_path = os.path.join(output_dir, "X_clinical_model_input.csv")
# 2. For the Genomic-Only Model
x_genomic_path = os.path.join(output_dir, "X_genomic_model_input.csv")
# 3. For the Combined (C+G) Model
x_combined_path = os.path.join(output_dir, "X_combined_model_input.csv")
# 4. The Target (y) for all models
y_target_path = os.path.join(output_dir, "y_survival_target.csv")

# --- Save the files ---
# (We use the variables we created at the end of Step 9)

# 1. Clinical-Only (60, 33)
X_clinical_processed_df_FULL.to_csv(x_clinical_path)

# 2. Genomic-Only (60, 2805)
X_genomic_final_FULL.to_csv(x_genomic_path)

# 3. Combined (C+G) (60, 2838)
X_final_FULL.to_csv(x_combined_path)

# 4. Target (60, 2)
y_survival_FULL.to_csv(y_target_path)

print(f"Success! Your data is saved properly for all models.")
print("\n--- For your 'Clinical-Only' model, use: ---")
print(f"X: {x_clinical_path}")
print(f"y: {y_target_path}")

print("\n--- For your 'Genomic-Only' model, use: ---")
print(f"X: {x_genomic_path}")
print(f"y: {y_target_path}")


--- Step 10: Saving final, model-ready data to disk... ---
Success! Your data is saved properly for all models.

--- For your 'Clinical-Only' model, use: ---
X: /Users/saifahmed/development/ESCC/data/processed/X_clinical_model_input.csv
y: /Users/saifahmed/development/ESCC/data/processed/y_survival_target.csv

--- For your 'Genomic-Only' model, use: ---
X: /Users/saifahmed/development/ESCC/data/processed/X_genomic_model_input.csv
y: /Users/saifahmed/development/ESCC/data/processed/y_survival_target.csv
